In [52]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import math as m
from collections import defaultdict

In [2]:
name_df=pd.read_csv('ranking_data_sets/name.basics.tsv', sep='\t')

In [3]:
name_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0069467,tt0050986,tt0050976"


In [53]:
top_names = name_df['primaryName'].value_counts().head(100)
top_names

Alex             456
Michael Smith    420
David Smith      409
Chris            409
Michael          408
                ... 
Mike Johnson     207
Andrew           206
Mike Brown       206
Daniel Smith     206
Richard Smith    206
Name: primaryName, Length: 100, dtype: int64

In [61]:
name_df[name_df['primaryName']=='Scarlett Johansson'].value_counts()

nconst     primaryName         birthYear  deathYear  primaryProfession            knownForTitles                         
nm0424060  Scarlett Johansson  1984       \N         actress,soundtrack,producer  tt0335266,tt1798709,tt1441395,tt0848228    1
dtype: int64

In [57]:
name_df[name_df['primaryName']=='Emma Mackey'].value_counts()

nconst     primaryName  birthYear  deathYear  primaryProfession  knownForTitles                           
nm8076281  Emma Mackey  1996       \N         actress            tt10366514,tt7657566,tt1517268,tt12374656    1
dtype: int64

In [58]:
name_df[name_df['primaryName']=='Margot Robbie'].value_counts()

nconst     primaryName    birthYear  deathYear  primaryProfession            knownForTitles                         
nm3053338  Margot Robbie  1990       \N         actress,producer,soundtrack  tt7713068,tt0993846,tt1386697,tt5580036    1
dtype: int64

In [59]:
name_df[name_df['primaryName']=='Johnny Depp'].value_counts()

nconst     primaryName  birthYear  deathYear  primaryProfession          knownForTitles                         
nm0000136  Johnny Depp  1963       \N         actor,producer,soundtrack  tt0325980,tt0449088,tt0408236,tt0367594    1
dtype: int64

In [67]:
name_df[name_df['primaryName']=='Jason Momoa'].value_counts()

nconst     primaryName  birthYear  deathYear  primaryProfession      knownForTitles                         
nm0597388  Jason Momoa  1979       \N         actor,producer,writer  tt5433140,tt0974015,tt1477834,tt4334266    1
dtype: int64

In [62]:
name_df[name_df['primaryName']=='Rinko Kikuchi'].value_counts()

nconst     primaryName    birthYear  deathYear  primaryProfession        knownForTitles                         
nm0452860  Rinko Kikuchi  1981       \N         actress,director,writer  tt0844286,tt1663662,tt0449467,tt3263614    1
dtype: int64

In [63]:
name_df[name_df['primaryName']=='Ben Kingsley'].value_counts()

nconst     primaryName   birthYear  deathYear  primaryProfession          knownForTitles                         
nm0001426  Ben Kingsley  1943       \N         actor,soundtrack,producer  tt0083987,tt0108052,tt0203119,tt1300854    1
dtype: int64

In [64]:
name_df[name_df['primaryName']=='Om Puri'].value_counts()

nconst     primaryName  birthYear  deathYear  primaryProfession                     knownForTitles                         
nm0700875  Om Puri      1950       2017       actor,miscellaneous,music_department  tt0111742,tt0166175,tt0116409,tt2980648    1
dtype: int64

In [115]:
popular_actors=['Scarlett Johansson',
                'Emma Mackey',
                'Margot Robbie',
                'Johnny Depp',
                'Jason Momoa',
                'Rinko Kikuchi',
                'Ben Kingsley',
                'Om Puri']


name_fa = name_df[name_df['primaryName'].isin(popular_actors)]
len(name_fa)

8

In [116]:
name_fa

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
135,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594"
1424,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer","tt0083987,tt0108052,tt0203119,tt1300854"
400915,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer","tt0335266,tt1798709,tt1441395,tt0848228"
427940,nm0452860,Rinko Kikuchi,1981,\N,"actress,director,writer","tt0844286,tt1663662,tt0449467,tt3263614"
563616,nm0597388,Jason Momoa,1979,\N,"actor,producer,writer","tt5433140,tt0974015,tt1477834,tt4334266"
660711,nm0700875,Om Puri,1950,2017,"actor,miscellaneous,music_department","tt0111742,tt0166175,tt0116409,tt2980648"
7421027,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036"
11563191,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656"


In [86]:
def append_titles(df, nconsts, new_titles):
    for nconst, new_title in zip(nconsts, new_titles):
        if nconst in df['nconst'].values:
            df.loc[df['nconst'] == nconst, 'knownForTitles'] = df.loc[df['nconst'] == nconst, 'knownForTitles'].apply(lambda x: x + ',' + new_title)
    return df


In [87]:
name_fa=append_titles(name_fa,['nm0424060','nm0001426'],['tt1219827','tt0107151'])

/var/folders/fc/2czwg5s15mvc98ksz0lc2bz80000gn/T/ipykernel_53930/3809709440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['nconst'] == nconst, 'knownForTitles'] = df.loc[df['nconst'] == nconst, 'knownForTitles'].apply(lambda x: x + ',' + new_title)


In [70]:
name_fa

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
135,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594"
1424,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer","tt0083987,tt0108052,tt0203119,tt1300854"
400915,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer","tt0335266,tt1798709,tt1441395,tt0848228"
427940,nm0452860,Rinko Kikuchi,1981,\N,"actress,director,writer","tt0844286,tt1663662,tt0449467,tt3263614"
563616,nm0597388,Jason Momoa,1979,\N,"actor,producer,writer","tt5433140,tt0974015,tt1477834,tt4334266"
660711,nm0700875,Om Puri,1950,2017,"actor,miscellaneous,music_department","tt0111742,tt0166175,tt0116409,tt2980648"
7421027,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036"
11563191,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656"


In [88]:
name_fa

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
135,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594"
1424,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer","tt0083987,tt0108052,tt0203119,tt1300854,tt0107151"
400915,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer","tt0335266,tt1798709,tt1441395,tt0848228,tt1219827"
427940,nm0452860,Rinko Kikuchi,1981,\N,"actress,director,writer","tt0844286,tt1663662,tt0449467,tt3263614"
563616,nm0597388,Jason Momoa,1979,\N,"actor,producer,writer","tt5433140,tt0974015,tt1477834,tt4334266"
660711,nm0700875,Om Puri,1950,2017,"actor,miscellaneous,music_department","tt0111742,tt0166175,tt0116409,tt2980648"
7421027,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036"
11563191,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656"


In [8]:
title_df=pd.read_csv('ranking_data_sets/title.ratings.tsv', sep='\t')

In [89]:
known_titles = name_fa['knownForTitles'].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
known_titles.name = 'knownForTitle'

expanded_name_df = name_fa.drop(columns=['knownForTitles']).join(known_titles)


merged_df = pd.merge(expanded_name_df, title_df, left_on='knownForTitle', right_on='tconst')


print(merged_df.head())

      nconst   primaryName birthYear deathYear          primaryProfession  \
0  nm0000136   Johnny Depp      1963        \N  actor,producer,soundtrack   
1  nm0000136   Johnny Depp      1963        \N  actor,producer,soundtrack   
2  nm0000136   Johnny Depp      1963        \N  actor,producer,soundtrack   
3  nm0000136   Johnny Depp      1963        \N  actor,producer,soundtrack   
4  nm0001426  Ben Kingsley      1943        \N  actor,soundtrack,producer   

  knownForTitle     tconst  averageRating  numVotes  
0     tt0325980  tt0325980            8.1   1186432  
1     tt0449088  tt0449088            7.1    681989  
2     tt0408236  tt0408236            7.3    382691  
3     tt0367594  tt0367594            6.7    513266  
4     tt0083987  tt0083987            8.0    238386  


In [90]:
merged_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitle,tconst,averageRating,numVotes
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0325980,tt0325980,8.1,1186432
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0449088,tt0449088,7.1,681989
2,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0408236,tt0408236,7.3,382691
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0367594,tt0367594,6.7,513266
4,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0083987,tt0083987,8.0,238386
5,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0108052,tt0108052,9.0,1422164
6,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0203119,tt0203119,7.3,65449
7,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt1300854,tt1300854,7.1,887185
8,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0107151,tt0107151,6.9,24548
9,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer",tt0335266,tt0335266,7.7,479601


In [21]:
title_basics_df=pd.read_csv('ranking_data_sets/title.basics.tsv', sep='\t')

/var/folders/fc/2czwg5s15mvc98ksz0lc2bz80000gn/T/ipykernel_53930/1245773564.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics_df=pd.read_csv('ranking_data_sets/title.basics.tsv', sep='\t')


In [84]:
filtered_tm=title_basics_df[title_basics_df['primaryTitle'].str.contains('Ghost in the Shell', case=False, na=False)]
filtered_tm[filtered_tm['titleType']=='movie']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
110984,tt0113568,movie,Ghost in the Shell,Kôkaku kidôtai,0,1995,\N,83,"Action,Animation,Crime"
332426,tt0347246,movie,Ghost in the Shell 2: Innocence,Innocence,0,2004,\N,100,"Animation,Drama,Mystery"
2186021,tt1219827,movie,Ghost in the Shell,Ghost in the Shell,0,2017,\N,107,"Action,Crime,Drama"
2402513,tt1260502,movie,Ghost in the Shell 2.0,Kôkaku kidôtai 2.0,0,2008,\N,83,"Action,Animation,Crime"
3907679,tt15392166,movie,Ghost in the Shell: SAC_2045 - Sustainable War,Kôkaku kidôtai SAC_2045 Jizoku kanô sensô,0,2021,\N,119,"Action,Animation,Sci-Fi"
4856261,tt1856022,movie,Ghost in the Shell S.A.C. Solid State Society 3D,Kôkaku kidôtai S.A.C. Solid State Society 3D,0,2011,\N,130,"Action,Animation,Fantasy"
6268424,tt2636124,movie,Ghost in the Shell: Arise - Border 1: Ghost Pain,Ghost in the Shell: Arise - Border 1: Ghost Pain,0,2013,\N,58,"Action,Animation,Sci-Fi"
7252811,tt3017864,movie,Ghost in the Shell: Arise - Border 2: Ghost Wh...,Ghost in the Shell: Arise - Border 2: Ghost Wh...,0,2013,\N,57,"Action,Animation,Sci-Fi"
7562909,tt3579524,movie,Ghost in the Shell: Arise - Border 3: Ghost Tears,Ghost in the Shell: Arise - Border 3: Ghost Tears,0,2014,\N,55,"Action,Animation,Sci-Fi"
7755505,tt4016942,movie,Ghost in the Shell: Arise - Border 4: Ghost St...,Ghost in the Shell: Arise - Border 4: Ghost St...,0,2014,\N,59,"Action,Animation,Sci-Fi"


In [82]:
title_basics_df[title_basics_df['primaryTitle'].str.contains('The House of the Spirits', case=False, na=False)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
104735,tt0107151,movie,The House of the Spirits,The House of the Spirits,0,1993,\N,145,"Drama,Romance"
8694599,tt6140480,tvEpisode,Jimmy Hollywood/Thumbelina/The House of the Sp...,Jimmy Hollywood/Thumbelina/The House of the Sp...,0,1994,\N,\N,Talk-Show
9848820,tt8686222,tvSeries,The House of The Spirits,The House of The Spirits,0,\N,\N,\N,Horror


In [91]:
merged_df2 = pd.merge(merged_df, title_basics_df, left_on='knownForTitle', right_on='tconst')

In [92]:
merged_df2

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitle,tconst_x,averageRating,numVotes,tconst_y,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0325980,tt0325980,8.1,1186432,tt0325980,movie,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,0,2003,\N,143,"Action,Adventure,Fantasy"
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0449088,tt0449088,7.1,681989,tt0449088,movie,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,0,2007,\N,169,"Action,Adventure,Fantasy"
2,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0408236,tt0408236,7.3,382691,tt0408236,movie,Sweeney Todd: The Demon Barber of Fleet Street,Sweeney Todd: The Demon Barber of Fleet Street,0,2007,\N,116,"Drama,Horror,Musical"
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0367594,tt0367594,6.7,513266,tt0367594,movie,Charlie and the Chocolate Factory,Charlie and the Chocolate Factory,0,2005,\N,115,"Adventure,Comedy,Family"
4,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0083987,tt0083987,8.0,238386,tt0083987,movie,Gandhi,Gandhi,0,1982,\N,191,"Biography,Drama,History"
5,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0108052,tt0108052,9.0,1422164,tt0108052,movie,Schindler's List,Schindler's List,0,1993,\N,195,"Biography,Drama,History"
6,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0203119,tt0203119,7.3,65449,tt0203119,movie,Sexy Beast,Sexy Beast,0,2000,\N,89,"Crime,Drama,Thriller"
7,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt1300854,tt1300854,7.1,887185,tt1300854,movie,Iron Man 3,Iron Man Three,0,2013,\N,130,"Action,Adventure,Sci-Fi"
8,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0107151,tt0107151,6.9,24548,tt0107151,movie,The House of the Spirits,The House of the Spirits,0,1993,\N,145,"Drama,Romance"
9,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer",tt0335266,tt0335266,7.7,479601,tt0335266,movie,Lost in Translation,Lost in Translation,0,2003,\N,102,"Comedy,Drama"


In [37]:
title_eps_df=pd.read_csv('ranking_data_sets/title.episode.tsv', sep='\t')

In [40]:
eps_df_selected=title_eps_df[['tconst','seasonNumber','episodeNumber']]

In [95]:
merged_df3 = pd.merge(merged_df2, eps_df_selected, left_on='tconst_x', right_on='tconst')

In [96]:
merged_df3

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitle,tconst_x,averageRating,numVotes,tconst_y,...,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,tconst,seasonNumber,episodeNumber


In [97]:
def transform_genres(df):
    df['genres'] = df['genres'].str.split(',')
    unique_genres = set(genre for sublist in df['genres'] for genre in sublist if type(sublist) == list)

    for genre in unique_genres:
        df[genre] = df['genres'].apply(lambda x: 1 if genre in x else 0)
    df.drop('genres', axis=1, inplace=True)

    return df

transformed_df = transform_genres(merged_df2.copy())


In [99]:
transformed_df.describe()

,averageRating,numVotes,Musical,Comedy,Biography,Horror,Fantasy,History,Action,Sci-Fi,Adventure,Thriller,Mystery,Family,Crime,Drama,Romance
count,34.000000,3.400000e+01,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,6.961765,4.104342e+05,0.029412,0.264706,0.176471,0.117647,0.176471,0.117647,0.382353,0.117647,0.352941,0.058824,0.088235,0.029412,0.176471,0.558824,0.088235
std,0.806795,4.373147e+05,0.171499,0.447811,0.386953,0.327035,0.386953,0.327035,0.493270,0.327035,0.485071,0.238833,0.287902,0.171499,0.386953,0.503995,0.287902
min,5.200000,6.993000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.300000,6.519550e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.900000,2.488500e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,7.450000,5.205380e+05,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,9.000000,1.535779e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [98]:
transformed_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitle,tconst_x,averageRating,numVotes,tconst_y,...,History,Action,Sci-Fi,Adventure,Thriller,Mystery,Family,Crime,Drama,Romance
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0325980,tt0325980,8.1,1186432,tt0325980,...,0,1,0,1,0,0,0,0,0,0
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0449088,tt0449088,7.1,681989,tt0449088,...,0,1,0,1,0,0,0,0,0,0
2,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0408236,tt0408236,7.3,382691,tt0408236,...,0,0,0,0,0,0,0,0,1,0
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack",tt0367594,tt0367594,6.7,513266,tt0367594,...,0,0,0,1,0,0,1,0,0,0
4,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0083987,tt0083987,8.0,238386,tt0083987,...,1,0,0,0,0,0,0,0,1,0
5,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0108052,tt0108052,9.0,1422164,tt0108052,...,1,0,0,0,0,0,0,0,1,0
6,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0203119,tt0203119,7.3,65449,tt0203119,...,0,0,0,0,1,0,0,1,1,0
7,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt1300854,tt1300854,7.1,887185,tt1300854,...,0,1,1,1,0,0,0,0,0,0
8,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer",tt0107151,tt0107151,6.9,24548,tt0107151,...,0,0,0,0,0,0,0,0,1,1
9,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer",tt0335266,tt0335266,7.7,479601,tt0335266,...,0,0,0,0,0,0,0,0,1,0


In [93]:
merged_df2_selected=merged_df2[['primaryName','tconst_x','averageRating',
                                'numVotes','titleType','primaryTitle','isAdult','runtimeMinutes']]

In [94]:
merged_df2_selected

,primaryName,tconst_x,averageRating,numVotes,titleType,primaryTitle,isAdult,runtimeMinutes
0,Johnny Depp,tt0325980,8.1,1186432,movie,Pirates of the Caribbean: The Curse of the Bla...,0,143
1,Johnny Depp,tt0449088,7.1,681989,movie,Pirates of the Caribbean: At World's End,0,169
2,Johnny Depp,tt0408236,7.3,382691,movie,Sweeney Todd: The Demon Barber of Fleet Street,0,116
3,Johnny Depp,tt0367594,6.7,513266,movie,Charlie and the Chocolate Factory,0,115
4,Ben Kingsley,tt0083987,8.0,238386,movie,Gandhi,0,191
5,Ben Kingsley,tt0108052,9.0,1422164,movie,Schindler's List,0,195
6,Ben Kingsley,tt0203119,7.3,65449,movie,Sexy Beast,0,89
7,Ben Kingsley,tt1300854,7.1,887185,movie,Iron Man 3,0,130
8,Ben Kingsley,tt0107151,6.9,24548,movie,The House of the Spirits,0,145
9,Scarlett Johansson,tt0335266,7.7,479601,movie,Lost in Translation,0,102


In [34]:
merged_df2_selected = merged_df2_selected.replace('\\N', np.nan)

final_merged_df_cleaned = merged_df2_selected.dropna(axis=0, how='any')

In [35]:
final_merged_df_cleaned

,primaryName,tconst_x,averageRating,numVotes,titleType,primaryTitle,isAdult,runtimeMinutes
0,Michael Johnson,tt0159163,8.1,42,tvSeries,The Expert,0,50
1,Michael Johnson,tt0057757,7.9,175,tvSeries,The Human Jungle,0,60
2,Michael Johnson,tt0159897,7.1,38,tvSeries,The Newcomers,0,30
3,Michael Johnson,tt0064030,7.4,7999,movie,Anne of the Thousand Days,0,145
4,Michael Johnson,tt0103460,6.5,2583,tvSeries,Kung Fu: The Legend Continues,0,60
...,...,...,...,...,...,...,...,...
1264,Chris Smith,tt4945528,5.9,246,movie,BearCity 3,0,116
1266,Michael Johnson,tt1587934,6.8,4212,tvSeries,American Ninja Warrior,0,40
1267,Michael Smith,tt8408218,5.2,748,movie,The Mercenary,0,91
1268,Michael Smith,tt2353113,8.6,46,tvSeries,Sea Rescue,0,30


In [46]:
columns_to_int = ['isAdult', 'runtimeMinutes', 'numVotes']

for column in columns_to_int:
    final_merged_df_cleaned[column] = pd.to_numeric(final_merged_df_cleaned[column], errors='coerce')
    final_merged_df_cleaned = final_merged_df_cleaned.dropna(subset=[column])
    final_merged_df_cleaned[column] = final_merged_df_cleaned[column].astype(int)

In [47]:
final_merged_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1038 entries, 0 to 1270
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   primaryName     1038 non-null   object 
 1   tconst_x        1038 non-null   object 
 2   averageRating   1038 non-null   float64
 3   numVotes        1038 non-null   int64  
 4   titleType       1038 non-null   object 
 5   primaryTitle    1038 non-null   object 
 6   isAdult         1038 non-null   int64  
 7   runtimeMinutes  1038 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 73.0+ KB


In [48]:
final_merged_df_cleaned

,primaryName,tconst_x,averageRating,numVotes,titleType,primaryTitle,isAdult,runtimeMinutes
0,Michael Johnson,tt0159163,8.1,42,tvSeries,The Expert,0,50
1,Michael Johnson,tt0057757,7.9,175,tvSeries,The Human Jungle,0,60
2,Michael Johnson,tt0159897,7.1,38,tvSeries,The Newcomers,0,30
3,Michael Johnson,tt0064030,7.4,7999,movie,Anne of the Thousand Days,0,145
4,Michael Johnson,tt0103460,6.5,2583,tvSeries,Kung Fu: The Legend Continues,0,60
...,...,...,...,...,...,...,...,...
1264,Chris Smith,tt4945528,5.9,246,movie,BearCity 3,0,116
1266,Michael Johnson,tt1587934,6.8,4212,tvSeries,American Ninja Warrior,0,40
1267,Michael Smith,tt8408218,5.2,748,movie,The Mercenary,0,91
1268,Michael Smith,tt2353113,8.6,46,tvSeries,Sea Rescue,0,30


In [49]:
final_merged_df_cleaned.describe()

,averageRating,numVotes,isAdult,runtimeMinutes
count,1038.000000,1.038000e+03,1038.000000,1038.000000
mean,6.545183,3.321909e+04,0.002890,75.844894
std,1.559934,1.711032e+05,0.053708,48.242497
min,1.000000,5.000000e+00,0.000000,1.000000
25%,5.700000,3.400000e+01,0.000000,44.000000
50%,6.800000,2.960000e+02,0.000000,82.000000
75%,7.700000,2.757500e+03,0.000000,97.000000
max,10.000000,2.830206e+06,1.000000,623.000000


In [50]:
final_merged_df_cleaned.to_csv('ranking_data_sets/cleaned_imdb_name.csv', index=False)

In [51]:
final_merged_df_cleaned['titleType'].unique()

array(['tvSeries', 'movie', 'tvMiniSeries', 'tvSpecial', 'tvMovie',
       'video', 'short', 'tvShort', 'videoGame', 'tvEpisode'],
      dtype=object)

In [100]:
transformed_df.to_csv('ranking_data_sets/imdb_movie_stars.csv', index=False)

In [101]:
principle_df=pd.read_csv('ranking_data_sets/title.principals.tsv', sep='\t')

In [104]:
principle_df

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
59692257,tt9916880,5,nm0584014,director,\N,\N
59692258,tt9916880,6,nm0996406,director,principal director,\N
59692259,tt9916880,7,nm1482639,writer,\N,\N
59692260,tt9916880,8,nm2586970,writer,books,\N


In [112]:
principle_df[(principle_df['nconst'] == 'nm8076281') & ((principle_df['category'] == 'actress')|((principle_df['category'] == 'actor')))]

,tconst,ordering,nconst,category,job,characters
8695403,tt10366514,2,nm8076281,actress,\N,"[""Adrienne Bourgès""]"
13474405,tt11761560,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
15362043,tt12374656,1,nm8076281,actress,\N,"[""Emily Brontë""]"
19531485,tt13945444,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
19531497,tt13945446,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
19531530,tt13945454,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
19531542,tt13945456,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
19642953,tt13986132,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
19642964,tt13986134,4,nm8076281,actress,\N,"[""Maeve Wiley""]"
22551423,tt14923652,4,nm8076281,actress,\N,"[""Maeve Wiley""]"


In [108]:
principle_df[principle_df['nconst']=='nm0000136']

,tconst,ordering,nconst,category,job,characters
750062,tt0087800,2,nm0000136,actor,\N,"[""Glen Lantz""]"
768024,tt0089839,2,nm0000136,actor,\N,"[""Jack""]"
790025,tt0092312,1,nm0000136,actor,\N,"[""Officer Tom Hanson""]"
852356,tt0099329,1,nm0000136,actor,\N,"[""Cry-Baby""]"
853825,tt0099487,1,nm0000136,actor,\N,"[""Edward Scissorhands""]"
...,...,...,...,...,...,...
57493251,tt9179096,3,nm0000136,actor,\N,"[""W. Eugene Smith""]"
57813303,tt9283916,3,nm0000136,archive_footage,\N,"[""Self""]"
58042389,tt9361052,5,nm0000136,archive_footage,\N,"[""Self""]"
59138704,tt9725138,1,nm0000136,archive_footage,\N,"[""Self""]"


In [113]:
def get_tconst(principle_df,nconst):
    df=principle_df[(principle_df['nconst'] == nconst) & ((principle_df['category'] == 'actress')|((principle_df['category'] == 'actor')))]
    return df['tconst'].tolist()

In [114]:
get_tconst(principle_df,'nm8076281')

['tt10366514',
 'tt11761560',
 'tt12374656',
 'tt13945444',
 'tt13945446',
 'tt13945454',
 'tt13945456',
 'tt13986132',
 'tt13986134',
 'tt14923652',
 'tt15494890',
 'tt17538632',
 'tt28941726',
 'tt28941739',
 'tt28941769',
 'tt28941778',
 'tt28941928',
 'tt28942002',
 'tt28942282',
 'tt30096221',
 'tt5642098',
 'tt7767422',
 'tt8431282',
 'tt8601954',
 'tt8601956',
 'tt8601960',
 'tt8601970',
 'tt8601972',
 'tt8601978',
 'tt8601980',
 'tt9067106',
 'tt9699184',
 'tt9699186',
 'tt9699190',
 'tt9699192',
 'tt9699194',
 'tt9699196',
 'tt9699202',
 'tt9699204',
 'tt9778348']

In [117]:
name_fa['films_tv_participated'] = name_fa['nconst'].apply(lambda x: get_tconst(principle_df, x))

/var/folders/fc/2czwg5s15mvc98ksz0lc2bz80000gn/T/ipykernel_53930/2512522344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_fa['films_tv_participated'] = name_fa['nconst'].apply(lambda x: get_tconst(principle_df, x))


In [119]:
name_fa['films_tv_participated'] = name_fa['films_tv_participated'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

/var/folders/fc/2czwg5s15mvc98ksz0lc2bz80000gn/T/ipykernel_53930/1594451471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_fa['films_tv_participated'] = name_fa['films_tv_participated'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


In [120]:
name_fa

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,films_tv_participated
135,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594","tt0087800, tt0089839, tt0092312, tt0099329, tt..."
1424,nm0001426,Ben Kingsley,1943,\N,"actor,soundtrack,producer","tt0083987,tt0108052,tt0203119,tt1300854","tt0072660, tt0072688, tt0072743, tt0073154, tt..."
400915,nm0424060,Scarlett Johansson,1984,\N,"actress,soundtrack,producer","tt0335266,tt1798709,tt1441395,tt0848228","tt0116985, tt0162346, tt0165396, tt0221799, tt..."
427940,nm0452860,Rinko Kikuchi,1981,\N,"actress,director,writer","tt0844286,tt1663662,tt0449467,tt3263614","tt0281214, tt0294130, tt0309703, tt0461394, tt..."
563616,nm0597388,Jason Momoa,1979,\N,"actor,producer,writer","tt5433140,tt0974015,tt1477834,tt4334266","tt0358203, tt0374455, tt0409594, tt0468534, tt..."
660711,nm0700875,Om Puri,1950,2017,"actor,miscellaneous,music_department","tt0111742,tt0166175,tt0116409,tt2980648","tt0080322, tt0080435, tt0083018, tt0083575, tt..."
7421027,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036","tt0900387, tt0993846, tt10304142, tt10640346, ..."
11563191,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656","tt10366514, tt11761560, tt12374656, tt13945444..."


In [124]:
title_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,2008
1,tt0000002,5.7,270
2,tt0000003,6.5,1925
3,tt0000004,5.4,178
4,tt0000005,6.2,2698
...,...,...,...
1380528,tt9916730,7.6,11
1380529,tt9916766,7.0,22
1380530,tt9916778,7.2,36
1380531,tt9916840,8.8,6


In [129]:
name_fa['films_tv_participated'] = name_fa['films_tv_participated'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

films_tv_participated = name_fa['films_tv_participated'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
films_tv_participated.name = 'films_tv_participated'

expanded_name_df = name_fa.drop(columns=['films_tv_participated']).join(films_tv_participated)

merged_df = pd.merge(expanded_name_df, title_df, left_on='films_tv_participated', right_on='tconst')

print(merged_df.head())


/var/folders/fc/2czwg5s15mvc98ksz0lc2bz80000gn/T/ipykernel_53930/895105726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_fa['films_tv_participated'] = name_fa['films_tv_participated'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


      nconst  primaryName birthYear deathYear          primaryProfession  \
0  nm0000136  Johnny Depp      1963        \N  actor,producer,soundtrack   
1  nm0000136  Johnny Depp      1963        \N  actor,producer,soundtrack   
2  nm0000136  Johnny Depp      1963        \N  actor,producer,soundtrack   
3  nm0000136  Johnny Depp      1963        \N  actor,producer,soundtrack   
4  nm0000136  Johnny Depp      1963        \N  actor,producer,soundtrack   

                            knownForTitles films_tv_participated     tconst  \
0  tt0325980,tt0449088,tt0408236,tt0367594             tt0087800  tt0087800   
1  tt0325980,tt0449088,tt0408236,tt0367594             tt0089839  tt0089839   
2  tt0325980,tt0449088,tt0408236,tt0367594             tt0092312  tt0092312   
3  tt0325980,tt0449088,tt0408236,tt0367594             tt0099329  tt0099329   
4  tt0325980,tt0449088,tt0408236,tt0367594             tt0099487  tt0099487   

   averageRating  numVotes  
0            7.4    258287  
1         

In [130]:
merged_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,films_tv_participated,tconst,averageRating,numVotes
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0087800,tt0087800,7.4,258287
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0089839,tt0089839,5.2,9060
2,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0092312,tt0092312,7.2,13769
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099329,tt0099329,6.5,63939
4,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099487,tt0099487,7.9,515557
...,...,...,...,...,...,...,...,...,...,...
796,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699194,tt9699194,7.7,4656
797,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699196,tt9699196,8.4,4969
798,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699202,tt9699202,7.8,6476
799,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699204,tt9699204,9.0,6794


In [132]:
merged_df2 = pd.merge(merged_df, title_basics_df, left_on='films_tv_participated', right_on='tconst')

In [133]:
merged_df2

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,films_tv_participated,tconst_x,averageRating,numVotes,tconst_y,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0087800,tt0087800,7.4,258287,tt0087800,movie,A Nightmare on Elm Street,A Nightmare on Elm Street,0,1984,\N,91,Horror
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0089839,tt0089839,5.2,9060,tt0089839,movie,Private Resort,Private Resort,0,1985,\N,82,Comedy
2,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0092312,tt0092312,7.2,13769,tt0092312,tvSeries,21 Jump Street,21 Jump Street,0,1987,1991,60,"Crime,Drama,Mystery"
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099329,tt0099329,6.5,63939,tt0099329,movie,Cry-Baby,Cry-Baby,0,1990,\N,85,"Comedy,Musical"
4,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099487,tt0099487,7.9,515557,tt0099487,movie,Edward Scissorhands,Edward Scissorhands,0,1990,\N,105,"Drama,Fantasy,Romance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699194,tt9699194,7.7,4656,tt9699194,tvEpisode,Episode 5,Episode 5,0,2020,\N,53,"Comedy,Drama,Romance"
797,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699196,tt9699196,8.4,4969,tt9699196,tvEpisode,Episode 6,Episode 6,0,2020,\N,54,"Comedy,Drama,Romance"
798,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699202,tt9699202,7.8,6476,tt9699202,tvEpisode,Episode 8,Episode 8,0,2020,\N,59,"Comedy,Drama,Romance"
799,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt9699204,tt9699204,9.0,6794,tt9699204,tvEpisode,Episode 7,Episode 7,0,2020,\N,49,"Comedy,Drama,Romance"


In [134]:
merged_df2_movies=merged_df2[merged_df2['titleType']=='movie']

In [138]:
transformed_df = transform_genres(merged_df2_movies.copy())

In [139]:
transformed_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,films_tv_participated,tconst_x,averageRating,numVotes,...,Documentary,Sport,Musical,Animation,Western,Mystery,History,Action,Adventure,War
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0087800,tt0087800,7.4,258287,...,0,0,0,0,0,0,0,0,0,0
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0089839,tt0089839,5.2,9060,...,0,0,0,0,0,0,0,0,0,0
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099329,tt0099329,6.5,63939,...,0,0,1,0,0,0,0,0,0,0
4,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099487,tt0099487,7.9,515557,...,0,0,0,0,0,0,0,0,0,0
5,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0106307,tt0106307,7.2,45213,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036",tt7131622,tt7131622,7.6,814002,...,0,0,0,0,0,0,0,0,0,0
761,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036",tt7713068,tt7713068,6.1,259314,...,0,0,0,0,0,0,0,1,0,0
763,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt10366514,tt10366514,6.2,6993,...,0,0,0,0,0,0,1,0,0,0
765,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt12374656,tt12374656,6.8,7390,...,0,0,0,0,0,0,1,0,0,0


In [146]:
transformed_df = transformed_df[transformed_df['runtimeMinutes'] != '\\N']


In [147]:
transformed_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,films_tv_participated,tconst_x,averageRating,numVotes,...,Documentary,Sport,Musical,Animation,Western,Mystery,History,Action,Adventure,War
0,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0087800,tt0087800,7.4,258287,...,0,0,0,0,0,0,0,0,0,0
1,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0089839,tt0089839,5.2,9060,...,0,0,0,0,0,0,0,0,0,0
3,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099329,tt0099329,6.5,63939,...,0,0,1,0,0,0,0,0,0,0
4,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0099487,tt0099487,7.9,515557,...,0,0,0,0,0,0,0,0,0,0
5,nm0000136,Johnny Depp,1963,\N,"actor,producer,soundtrack","tt0325980,tt0449088,tt0408236,tt0367594",tt0106307,tt0106307,7.2,45213,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036",tt7131622,tt7131622,7.6,814002,...,0,0,0,0,0,0,0,0,0,0
761,nm3053338,Margot Robbie,1990,\N,"actress,producer,soundtrack","tt7713068,tt0993846,tt1386697,tt5580036",tt7713068,tt7713068,6.1,259314,...,0,0,0,0,0,0,0,1,0,0
763,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt10366514,tt10366514,6.2,6993,...,0,0,0,0,0,0,1,0,0,0
765,nm8076281,Emma Mackey,1996,\N,actress,"tt10366514,tt7657566,tt1517268,tt12374656",tt12374656,tt12374656,6.8,7390,...,0,0,0,0,0,0,1,0,0,0


In [148]:
transformed_df.to_csv('ranking_data_sets/imdb_movie_stars.csv', index=False)